In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# =============================================================================
# IMPORT LIBRARIES
# =============================================================================
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import classification_report, accuracy_score
from PIL import Image
import os

device = torch.device("cpu")


# =============================================================================
# DEFINE MODEL ARCHITECTURE
# =============================================================================
class AQIClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(AQIClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        
        self.fc4 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout2(out)

        out = self.fc3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        
        out = self.fc4(out)
        return out


# =============================================================================
# LOAD MODEL AND PREPROCESSING OBJECTS
# =============================================================================
print("Loading model and preprocessing objects...")

scaler = joblib.load('scaler.pkl')
label_encoder = joblib.load('label_encoder.pkl')

input_size = scaler.n_features_in_
hidden_size1 = 128
hidden_size2 = 128
hidden_size3 = 128
output_size = len(label_encoder.classes_)

model = AQIClassifier(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)
model.load_state_dict(torch.load('aqi_classifier.pth', map_location=device))
model.eval()
model.to(device)

print("Model loaded successfully!")

Loading model and preprocessing objects...
Model loaded successfully!


c:\Users\ASUS\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ASUS\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# ĐÁNH GIÁ MÔ HÌNH VÀ XỬ LÝ TIF FILES

In [4]:
# =============================================================================
# ĐÁNH GIÁ MÔ HÌNH TRÊN TẬP TEST (DỮ LIỆU CSV)
# =============================================================================

print("="*70)
print("ĐÁNH GIÁ MÔ HÌNH AQI CLASSIFICATION TRÊN TẬP TEST")
print("="*70)

# Đọc dữ liệu test
csv_file = 'data_onkk (2).csv'
df = pd.read_csv(csv_file)
print(f"\nĐã load {len(df)} dòng dữ liệu test từ {csv_file}")

# Tạo AQI_Class từ pm25
def pm25_to_aqi_class(pm25):
    if pm25 <= 15.4:
        return 'Tốt'
    elif pm25 <= 40.4:
        return 'Trung bình'
    elif pm25 <= 65.4:
        return 'Kém'
    elif pm25 <= 150.4:
        return 'Xấu'
    else:
        return 'Rất xấu'

df['AQI_Class'] = df['pm25'].apply(pm25_to_aqi_class)

# Phân bố class trong tập test
print("\nPhân bố AQI class trong tập test:")
print(df['AQI_Class'].value_counts().sort_index())

# Chuẩn bị features và labels
feature_columns = ['PRES2M', 'RH', 'WSPD', 'TMP', 'TP', 'SQRT_SEA_DEM_LAT']
X_test = df[feature_columns]
y_test = df['AQI_Class']

# Chuẩn hóa features
X_test_scaled = scaler.transform(X_test)

# Chuyển sang tensor
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)

# Dự đoán
print("\nĐang dự đoán trên tập test...")
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

# Chuyển đổi predictions về labels
y_pred = label_encoder.inverse_transform(predicted.cpu().numpy())

# =============================================================================
# CÁC CHỈ SỐ ĐÁNH GIÁ PHÂN LỚP
# =============================================================================
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                            f1_score, confusion_matrix, classification_report)

print("\n" + "="*70)
print("CÁC CHỈ SỐ ĐÁNH GIÁ PHÂN LỚP")
print("="*70)

# 1. Accuracy (Độ chính xác tổng thể)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n1. ACCURACY (Độ chính xác): {accuracy:.4f} ({accuracy*100:.2f}%)")
print("   → Tỷ lệ dự đoán đúng trên tổng số mẫu test")

# 2. Precision (Độ chính xác của từng class)
precision_macro = precision_score(y_test, y_pred, average='macro', zero_division=0)
precision_weighted = precision_score(y_test, y_pred, average='weighted', zero_division=0)
print(f"\n2. PRECISION (Độ chính xác từng class):")
print(f"   - Macro avg: {precision_macro:.4f}")
print(f"   - Weighted avg: {precision_weighted:.4f}")
print("   → Trong các mẫu được dự đoán là class X, bao nhiêu % thực sự là X")

# 3. Recall (Độ phủ - Khả năng tìm ra đúng)
recall_macro = recall_score(y_test, y_pred, average='macro', zero_division=0)
recall_weighted = recall_score(y_test, y_pred, average='weighted', zero_division=0)
print(f"\n3. RECALL (Độ phủ):")
print(f"   - Macro avg: {recall_macro:.4f}")
print(f"   - Weighted avg: {recall_weighted:.4f}")
print("   → Trong tất cả mẫu thực sự là class X, mô hình tìm ra được bao nhiêu %")

# 4. F1-Score (Trung bình điều hòa của Precision và Recall)
f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)
f1_weighted = f1_score(y_test, y_pred, average='weighted', zero_division=0)
print(f"\n4. F1-SCORE (Điểm cân bằng):")
print(f"   - Macro avg: {f1_macro:.4f}")
print(f"   - Weighted avg: {f1_weighted:.4f}")
print("   → Điểm cân bằng giữa Precision và Recall (F1 = 2 * P * R / (P + R))")

# 5. Classification Report chi tiết
print("\n" + "="*70)
print("CLASSIFICATION REPORT CHI TIẾT (Precision, Recall, F1 từng class)")
print("="*70)
report = classification_report(y_test, y_pred, zero_division=0)
print(report)

# 6. Confusion Matrix
print("="*70)
print("CONFUSION MATRIX (Ma trận nhầm lẫn)")
print("="*70)
cm = confusion_matrix(y_test, y_pred, labels=label_encoder.classes_)
print("\nClasses:", list(label_encoder.classes_))
print("\nMatrix (hàng = thực tế, cột = dự đoán):")
print(cm)

# Tính tỷ lệ dự đoán đúng từng class
print("\nTỷ lệ dự đoán đúng từng class:")
for i, class_name in enumerate(label_encoder.classes_):
    total = cm[i].sum()
    correct = cm[i, i]
    if total > 0:
        print(f"  {class_name}: {correct}/{total} = {correct/total*100:.1f}%")

# =============================================================================
# KẾT LUẬN VÀ LƯU KẾT QUẢ
# =============================================================================
print("\n" + "="*70)
print("KẾT LUẬN")
print("="*70)
print(f"\n✓ Mô hình AQI Classification đạt Accuracy = {accuracy*100:.2f}%")
print(f"✓ F1-Score (weighted) = {f1_weighted:.4f}")
print(f"✓ Precision (weighted) = {precision_weighted:.4f}")
print(f"✓ Recall (weighted) = {recall_weighted:.4f}")

if accuracy >= 0.60:
    print("\n→ MÔ HÌNH TỐT: Accuracy >= 60%, đạt yêu cầu cho việc tạo bản đồ dự báo")
elif accuracy >= 0.50:
    print("\n→ MÔ HÌNH TRUNG BÌNH: Accuracy 50-60%, cần cải thiện nhưng có thể sử dụng")
else:
    print("\n→ MÔ HÌNH CẦN CẢI THIỆN: Accuracy < 50%, nên xem xét fine-tuning")

# Lưu report chi tiết
report_file = 'output_reports/classification_report_notebook.txt'
with open(report_file, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("ĐÁNH GIÁ MÔ HÌNH AQI CLASSIFICATION TRÊN TẬP TEST\n")
    f.write("="*70 + "\n\n")
    f.write(f"Số mẫu test: {len(df)}\n\n")
    f.write("CÁC CHỈ SỐ ĐÁNH GIÁ PHÂN LỚP:\n")
    f.write("-"*70 + "\n")
    f.write(f"1. Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)\n")
    f.write(f"2. Precision (weighted): {precision_weighted:.4f}\n")
    f.write(f"3. Recall (weighted): {recall_weighted:.4f}\n")
    f.write(f"4. F1-Score (weighted): {f1_weighted:.4f}\n\n")
    f.write("="*70 + "\n")
    f.write("CLASSIFICATION REPORT CHI TIẾT\n")
    f.write("="*70 + "\n")
    f.write(report)
    f.write("\n" + "="*70 + "\n")
    f.write("CONFUSION MATRIX\n")
    f.write("="*70 + "\n")
    f.write(f"Classes: {list(label_encoder.classes_)}\n\n")
    f.write(str(cm))

print(f"\n✓ Đã lưu report chi tiết: {report_file}")
print("\n→ Sử dụng mô hình này để tạo bản đồ dự báo AQI trong cell tiếp theo")


ĐÁNH GIÁ MÔ HÌNH AQI CLASSIFICATION TRÊN TẬP TEST

Đã load 1804 dòng dữ liệu test từ data_onkk (2).csv

Phân bố AQI class trong tập test:
AQI_Class
Kém           425
Rất xấu         9
Trung bình    951
Tốt           251
Xấu           168
Name: count, dtype: int64

Đang dự đoán trên tập test...

CÁC CHỈ SỐ ĐÁNH GIÁ PHÂN LỚP

1. ACCURACY (Độ chính xác): 0.5516 (55.16%)
   → Tỷ lệ dự đoán đúng trên tổng số mẫu test

2. PRECISION (Độ chính xác từng class):
   - Macro avg: 0.6730
   - Weighted avg: 0.6479
   → Trong các mẫu được dự đoán là class X, bao nhiêu % thực sự là X

3. RECALL (Độ phủ):
   - Macro avg: 0.7122
   - Weighted avg: 0.5516
   → Trong tất cả mẫu thực sự là class X, mô hình tìm ra được bao nhiêu %

4. F1-SCORE (Điểm cân bằng):
   - Macro avg: 0.6493
   - Weighted avg: 0.5538
   → Điểm cân bằng giữa Precision và Recall (F1 = 2 * P * R / (P + R))

CLASSIFICATION REPORT CHI TIẾT (Precision, Recall, F1 từng class)
              precision    recall  f1-score   support

         

c:\Users\ASUS\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [14]:
# =============================================================================
# XỬ LÝ TIF FILES VÀ TẠO BẢN ĐỒ AQI CHO TẤT CẢ CÁC NGÀY
# =============================================================================
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

print("\n" + "="*70)
print("XỬ LÝ TIF FILES VÀ TẠO BẢN ĐỒ AQI")
print("="*70)

# Đường dẫn thư mục chứa TIF files
feature_maps_dir = "Feature_Maps-20251116T094941Z-1-001/Feature_Maps"

# Lấy danh sách các ngày
feature_names = ['PRES2M', 'RH', 'WSPD', 'TMP', 'TP']
sample_dir = os.path.join(feature_maps_dir, feature_names[0])
available_dates = sorted([f.replace(f'{feature_names[0]}_', '').replace('.tif', '') 
                         for f in os.listdir(sample_dir) if f.endswith('.tif')])

print(f"\nCác ngày có sẵn: {available_dates}")
print(f"Sẽ xử lý {len(available_dates)} ngày")

# Đọc SQRT_SEA_DEM_LAT (giống nhau cho tất cả ngày)
sqrt_file = os.path.join(feature_maps_dir, "SQRT_SEA_DEM_LAT.tif")
img = Image.open(sqrt_file)
sqrt_feature = np.array(img)

# Thiết lập màu sắc cho AQI
colors_list = ['#00E400', '#FFFF00', '#FF7E00', '#FF0000', '#8F3F97']
cmap = mcolors.ListedColormap(colors_list)
bounds = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)
aqi_value_map = {'Tốt': 1, 'Trung bình': 2, 'Kém': 3, 'Xấu': 4, 'Rất xấu': 5}

# Xử lý từng ngày
all_predictions = []

for date_to_process in available_dates:
    print(f"\n{'='*70}")
    print(f"Xử lý ngày: {date_to_process}")
    print('='*70)
    
    # Đọc các TIF files
    features = {}
    for feature_name in feature_names:
        file_path = os.path.join(feature_maps_dir, feature_name, f"{feature_name}_{date_to_process}.tif")
        img = Image.open(file_path)
        features[feature_name] = np.array(img)
        height, width = features[feature_name].shape
    
    features['SQRT_SEA_DEM_LAT'] = sqrt_feature
    
    # Ghép features
    feature_stack = np.stack([
        features['PRES2M'],
        features['RH'],
        features['WSPD'],
        features['TMP'],
        features['TP'],
        features['SQRT_SEA_DEM_LAT']
    ], axis=-1)
    
    num_pixels = height * width
    feature_matrix = feature_stack.reshape(num_pixels, 6)
    
    # Lọc pixel hợp lệ
    valid_mask = ~np.isnan(feature_matrix).any(axis=1)
    nodata_mask = (feature_matrix == -9999.0).any(axis=1)
    valid_mask = valid_mask & ~nodata_mask
    
    valid_features = feature_matrix[valid_mask]
    valid_indices = np.where(valid_mask)[0]
    
    print(f"Kích thước: {height}x{width}, Pixel hợp lệ: {len(valid_features)} ({100*len(valid_features)/num_pixels:.1f}%)")
    
    # Dự đoán AQI
    scaled_features = scaler.transform(valid_features)
    input_tensor = torch.FloatTensor(scaled_features).to(device)
    
    with torch.no_grad():
        raw_output = model(input_tensor)
        _, predicted_indices = torch.max(raw_output, 1)
    
    predicted_labels = label_encoder.inverse_transform(predicted_indices.cpu().numpy())
    
    # Tạo DataFrame
    df_date = pd.DataFrame({
        'date': date_to_process,
        'pixel_index': valid_indices,
        'row': valid_indices // width,
        'col': valid_indices % width,
        'PRES2M': valid_features[:, 0],
        'RH': valid_features[:, 1],
        'WSPD': valid_features[:, 2],
        'TMP': valid_features[:, 3],
        'TP': valid_features[:, 4],
        'SQRT_SEA_DEM_LAT': valid_features[:, 5],
        'predicted_AQI': predicted_labels
    })
    
    all_predictions.append(df_date)
    
    # Lưu CSV
    output_csv = f'output_csv/TIF_Predictions_{date_to_process}.csv'
    df_date.to_csv(output_csv, index=False)
    
    # Phân bố AQI
    aqi_dist = df_date['predicted_AQI'].value_counts().sort_index()
    print(f"Phân bố AQI: {', '.join([f'{k}={v}' for k, v in aqi_dist.items()])}")
    
    # TẠO BẢN ĐỒ
    print(f"Tạo bản đồ...")
    aqi_map = np.full((height, width), np.nan)
    
    for _, row_data in df_date.iterrows():
        row_idx = int(row_data['row'])
        col_idx = int(row_data['col'])
        aqi_map[row_idx, col_idx] = aqi_value_map[row_data['predicted_AQI']]
    
    # Vẽ
    fig, ax = plt.subplots(figsize=(12, 10))
    im = ax.imshow(aqi_map, cmap=cmap, norm=norm, aspect='auto')
    
    cbar = plt.colorbar(im, ax=ax, boundaries=bounds, ticks=[1, 2, 3, 4, 5])
    cbar.ax.set_yticklabels(['Tốt', 'Trung bình', 'Kém', 'Xấu', 'Rất xấu'])
    cbar.set_label('AQI Class', rotation=270, labelpad=20, fontsize=12)
    
    date_formatted = f"{date_to_process[:4]}-{date_to_process[4:6]}-{date_to_process[6:8]}"
    ax.set_title(f'Bản đồ AQI từ dữ liệu TIF - {date_formatted}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Kinh độ (pixel)', fontsize=12)
    ax.set_ylabel('Vĩ độ (pixel)', fontsize=12)
    
    info_text = f"Pixel hợp lệ: {len(df_date):,}/{num_pixels:,} ({100*len(df_date)/num_pixels:.1f}%)"
    ax.text(0.02, 0.98, info_text, transform=ax.transAxes, fontsize=10, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    
    output_image = f'output_images/AQI_Map_{date_to_process}.png'
    plt.savefig(output_image, dpi=150, bbox_inches='tight')
    print(f"Đã lưu: {output_csv} và {output_image}")
    
    plt.close()

# Lưu tổng hợp
df_predictions = pd.concat(all_predictions, ignore_index=True)
df_predictions.to_csv('output_csv/TIF_Predictions_All_Dates.csv', index=False)

print("\n" + "="*70)
print("HOÀN THÀNH!")
print("="*70)
print(f"Đã xử lý {len(available_dates)} ngày")
print(f"Tổng số dòng dữ liệu: {len(df_predictions):,}")
print(f"Đã tạo {len(available_dates)} file CSV và {len(available_dates)} ảnh bản đồ")


XỬ LÝ TIF FILES VÀ TẠO BẢN ĐỒ AQI

Các ngày có sẵn: ['20200101', '20200529', '20200721', '20200810', '20201015', '20201215']
Sẽ xử lý 6 ngày

Xử lý ngày: 20200101
Kích thước: 81x73, Pixel hợp lệ: 2914 (49.3%)
Phân bố AQI: Kém=155, Rất xấu=218, Trung bình=501, Tốt=80, Xấu=1960
Tạo bản đồ...
Đã lưu: output_csv/TIF_Predictions_20200101.csv và output_images/AQI_Map_20200101.png

Xử lý ngày: 20200529
Kích thước: 81x73, Pixel hợp lệ: 2914 (49.3%)
Phân bố AQI: Kém=984, Trung bình=1704, Tốt=226
Tạo bản đồ...
Đã lưu: output_csv/TIF_Predictions_20200101.csv và output_images/AQI_Map_20200101.png

Xử lý ngày: 20200529
Kích thước: 81x73, Pixel hợp lệ: 2914 (49.3%)
Phân bố AQI: Kém=984, Trung bình=1704, Tốt=226
Tạo bản đồ...
Đã lưu: output_csv/TIF_Predictions_20200529.csv và output_images/AQI_Map_20200529.png

Xử lý ngày: 20200721
Kích thước: 81x73, Pixel hợp lệ: 2914 (49.3%)
Phân bố AQI: Kém=237, Trung bình=1163, Tốt=1514
Tạo bản đồ...
Đã lưu: output_csv/TIF_Predictions_20200529.csv và output_imag